# Dense Retrieval Passage for Open Domain Question & Answering
Submitted by: Rishabh Kaushick
<br>Date: March 16, 2025

# 1. Environment Setup

In [1]:
%pip install torch transformers faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [2]:
# installing datasets (hugging face dataset library)
%pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
from datasets import load_dataset

/opt/homebrew/anaconda3/envs/csye7230/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## CUDA: GPU Availability

In [3]:
 
print(f"PyTorch version: {torch.__version__}")
print(f"Is CUDA available: {torch.cuda.is_available()}")
 
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"cuDNN version: {torch.backends.cudnn.version()}")
    print(f"cuDNN enabled: {torch.backends.cudnn.enabled}")
    print(f"GPU Count: {torch.cuda.device_count()}")
    
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"Memory Allocated: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB")
        print(f"Memory Reserved: {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB")
 
else:
    print("CUDA is not available. Running on CPU.")

PyTorch version: 2.6.0
Is CUDA available: False
CUDA is not available. Running on CPU.


# 2. Dataset Selection and Exploration (15%)

## Loading Dataset (from HuggingFace)

### Download Training Set

In [5]:
dataset = load_dataset("ms_marco", "v2.1", split="train[:15000]")

Generating test split: 100%|██████████| 101092/101092 [00:00<00:00, 309588.97 examples/s]


### Saving Training Set to Disk

In [ ]:
# need to save the dataset in /data/train
dataset.save_to_disk("./data/ms_marco_15k")

### Loading Train Set from Disk

In [6]:
# https://huggingface.co/docs/datasets/v3.3.2/en/package_reference/main_classes#datasets.Dataset.save_to_disk

# load the already saved dataset from the disk
dataset.load_from_disk("./data/ms_marco_15k")

Saving the dataset (1/1 shards): 100%|██████████| 15000/15000 [00:00<00:00, 90952.61 examples/s]


Dataset({
    features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
    num_rows: 15000
})

In [7]:
print(len(dataset))  # This should show 15000

15000


### Loading & Saving Validation Set

In [ ]:
validation_dataset = load_dataset("ms_marco", "v2.1", split="validation")

In [ ]:
# save validation dataset
validation_dataset.save_to_disk("./data/validation/ms_marco")
validation_dataset.load_from_disk("./data/validation/ms_marco")

### Loading & Saving Test Set